Name: Deniz Ekiz

ID: 2016700051

## Homework 3 Programming Part

### Question
In this exercise we model a string of text using a Markov(1) model. For simplicity we only consider
letters ’a-z’. Capital letters ’A-Z’ are mapped to the corresponding ones. All remaining letters,
symbols, numbers, including spaces, are denoted by ’.’.
We have a probability table T where Ti,j = p(xt = j|xt−1 = i) transition model of letters in
English text for t = 1, 2 . . . N. Assume that the initial letter in a string is always a space denoted
as x0 = ’.’. Such a model where the probability table is always the same is sometimes called a
stationary model.


4. Discuss how you can improve the model to get better estimations



In [1]:
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

Hint: The code below loads a table of transition probabilities for English text.

In [2]:
import csv
import numpy as np
from IPython.display import display, Latex

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

print('Example')
## p(x_t = 'u' | x_{t-1} = 'q')
display(Latex(r"$p(x_t = \text{'u'} | x_{t-1} = \text{'q'})$"))
print(T[letter2idx['q']][letter2idx['u']])
display(Latex(r"$p(x_t | x_{t-1} = \text{'a'})$"))
t = 0.0
for c,p in zip(alphabet,T[letter2idx['a']]):
    print(c,p)

Example


<IPython.core.display.Latex object>

0.9949749


<IPython.core.display.Latex object>

('a', '0.0002835')
('b', '0.0228302')
('c', '0.0369041')
('d', '0.0426290')
('e', '0.0012216')
('f', '0.0075739')
('g', '0.0171385')
('h', '0.0014659')
('i', '0.0372661')
('j', '0.0002353')
('k', '0.0110124')
('l', '0.0778259')
('m', '0.0260757')
('n', '0.2145354')
('o', '0.0005459')
('p', '0.0195213')
('q', '0.0001749')
('r', '0.1104770')
('s', '0.0934290')
('t', '0.1317960')
('u', '0.0098029')
('v', '0.0306574')
('w', '0.0088799')
('x', '0.0009562')
('y', '0.0233701')
('z', '0.0018701')
('.', '0.0715219')


# Part 1
 For a given N, write a program to sample random strings with letters x1, x2, . . . , xN from p(x1:N|x0)

For N = 1 it is trivial. We will just need to select the corresponding section of conditional probability table.  Then make a random choice using np.random.choice function or argmax function.

$p(x_1|x_0 = \text{'.'})$

In [3]:
N = 1
#convert the string matrix to float. This is one time operation. Otherwise, we cant make calculations.
T = [[float(y) for y in x] for x in T]
# Assign first column of conditional probability table to a variable called guess_table
#letter2idx is key-value pair. It stores the corresponding value for each letter.
#Ex: letter2idx['a'] -> 0
#  letter2idx['b'] -> 1 , letter2idx['.'] -> 26
guess_table = T[letter2idx['.']]
print(np.random.choice(alphabet,1,p=guess_table))

['w']



We need to find what is $p(x_{1:N} | x_0)$

$p(x_{1:N} | x_0=\text{'.'})=\sum_{x_1,x_2,....,X_N}p(x_1|x_0 = \text{'.'})p(x_2|x_1)p(x_3|x_2)...p(x_{N}|x_{N-1}) $

We start selecting letter by letter we need to update the table each time.


Given x_0 = '.', 27X27 conditional probability table T and number of letters N.

Output, guess_table -> 27xN probability table which contains every probability of states x1 x2, ... xn with respect to letters
However, when we decide to likelihood function. We need to update each probability,

Algorithm should be, 

i = 0

Start with p(x1|x_0 ='.'), make your guess with respect to likelihood function. Then, update next tables.

return strings

For this case it is also trivial.

In [4]:
#given N
#create CDF
sn = 10
selection = 26 #string index in alphabet
def string_generator(selection,sn):
    N = 27
    selected_strings = np.zeros([0,1])
    for z in range(0,sn):
        CDF = np.zeros((0,2))
        my_sum = 0.0
        prev_let = alphabet[selection]
        for i in range(0,N):
            #my_str = "$p(x_t = \text{'" + alphabet[i]+ "'} | x_{t-1} = \text{'.'}$ ) = "+ T[i][letter2idx[prev_let]] 
            #display( Latex(r"$p(x_t = \text{'" + alphabet[i]+ r"'} | x_{t-1} = \text{'"+prev_let+r"'}$ ) = "+ T[i][letter2idx[prev_let]] ))
            CDF = np.vstack((CDF,[alphabet[i],float(T[i][letter2idx[prev_let]]) ]))
            my_sum += float(T[i][letter2idx[prev_let]])
        #print(my_sum)
        # normalize and find CDF
        CDF[0,1] = float(CDF[0,1])/my_sum
        for i in range(1,N):
            if(len(CDF[:,1])>1):
                CDF[i,1] =  float(CDF[i,1])/my_sum  + float(CDF[i-1,1])
            else:
                CDF[0,1] = 1


        #print(CDF)
        s = np.random.uniform(0,1,1)
        for i in range(0,len(s)):
            for j in range(0,len(CDF[:,1])):
                if(s[i] <= float(CDF[j,1])):
                    selected_strings = np.vstack((selected_strings,CDF[j,0]))
                    selection = j
                    break
    #print("For N= "+ str(N) +" and Sample number= "+ str(sn) +" Selected strings:")
    return selected_strings

#Select your N 
N = 1000
gens = string_generator(letter2idx['.'],N)
for i in range(0,len(gens)):
    print(str(gens[i,0])),

d n o p p x x e h c x e k a m a i d i f . u q x e d e r o k i l u q x e v x i d n o n o p a h c . r t s a p p y m h . e v e v l p p x x e b . o h c x x e r o s b . g . k c o d a g . g . y a e . d v o t p x e b b u q a w a z z z z z g n i r e h t i n e b b . y b a z z z z z z z u q x e b a z z z z r f o i p x x e r p s . h w o g o p o l a w o z u q o s k s y r e u q x x i a w e v d n a z a g n a v o z z z r b . y b u q x x e v i l b o j b u j t l u b y a m . y l l i p x e x x e u m r e n y z z z a h c n e i l a r e d e z z z r u q x e z z u o h c i w o s u q i p x e s u q c u q n a h g n i p e f e p m i f o j n k a z z r u q c x i v a g n a f o j e h w . g a m u q e r u q e v i b o . e l l l a . c s e b u q x x e v o n i g . g u q c c x x a w o l l u q . y a z a j b . a h . d n i l a . k c u q e s s l i w . s u q s t k c x e d e f o v o i x o r u q . d n o m a d n k n o i e w o t s e r f f u q x x i h t y b i w o m e r e t u q x x x x x i f o j b . s u j b . d n k n e c x e b o p m . d 

## Part 2
Question: Now suppose you are given strings with missing letters, where each missing letter is denoted
by a question mark (or underscore, as below). Implement a method, that samples missing
letters conditioned on observed ones, i.e., samples from p(x−α|xα) where α denotes indices
of observed letters. For example, if the input is ’t??.’, we have N = 4 and x1 = ’t’ and
x4 = ’.’, α = {1, 4} and −α = {2, 3}. Your program may possibly generate the strings ’the.’,
’twi.’, ’tee.’, etc. Hint: make sure to make use all data given and sample from the correct
distribution. Implement the method and print the results for the test strings below.

We have sucessfully generated our forward markov(1) model. Now let's turn it into a function

In [10]:
for i in range(0,10):
    print(" ")
    gens = string_generator(letter2idx['t'],3)
    print('t'),
    for i in range(0,len(gens)):
        print(str(gens[i,0])),

 
t c c o  
t c s u  
t s r o  
t o v o  
t r f f  
t a m o  
t u b .  
t x x x  
t u q .  
t c a f


In [21]:
def straight_markov(first_char,N,T,repeat,start,p,l):
    #27xN shaped matrix
    # given j  z
    guess_table = np.zeros((27,N+2))
    x_0 = first_char
    x_prev = x_0
    #fill guess_table
    word = first_char
    for j in range(0,N):
        for i in range(0,len(alphabet)):
            guess_table[i,0] = T[letter2idx[x_prev]][i]
        #normalize
        guess_table[:,0] = np.divide(guess_table[:,0],np.sum(guess_table[:,0]))
        if not l:
            x_prev = str(np.random.choice(alphabet,1,p = guess_table[:,0])[0])
            word = np.vstack((word ,x_prev))
        else:
            x_prev = str(alphabet[np.argmax(guess_table[:,0])])
            word = np.vstack((word ,x_prev))
    if p :
        print(''.join(word[start:,0]))
    else:
        return word[start:]
for i in range(0,10):
    straight_markov('t',3,T,10,0,p=True,l=False)

toni
t.me
tush
tthe
t.be
tha.
toba
tthe
t.ch
t.ho


The markov model is like x0 -> x1 -> x2-> .... xn -> xn+1

Let's look at for a _ _ b for getting some intuition. This model is p(x2,x3|x1=a,x4=b). We need to construct our table using these facts.

$p(x_2,x_3|x_1=\text{'a'}, x_4=b)$ $\alpha$ $p(x_4=\text{'b'},x_3,x_2,x_1=\text{'a'}) $

$=p(x_4=\text{'b'}|x_3,x_2,x_1=\text{'a'})p(x_3,x_2,x_1=\text{'a'}) $

$=p(x_4=\text{'b'}|x_3,x_2,x_1='a')p(x_3|x_2,x_1='a')p(x_2,x_1='a')$

$=p(x_4=\text{'b'}|x_3,x_2,x_1=\text{'a'})p(x_3|x_2,x_1=\text{'a'})p(x_2|x_1=\text{'a'})p(x_1=\text{'a'}|x_o = \text{'.'})$

There is also x0 = '.' in this model, since we know that x0 = '.' 

p(x0 = '.') = 1








Applying conditional independences the equation turns out to be,

$= p(x_4=\text{'b'}|x_3)p(x_3|x_2)p(x_2|x_1 = \text{'a'})p(x_1 = \text{'a'}|x_0 = \text{'.'})$

$p(x_1 = 'a'|x_0 ='.')$ is a scalar.

$p(x_2|x_1=\text{'a'})$ is vector.

$p(x_3|x_2)$ is our probability table T.

$p(x4=\text{'b'}|x3)$ is a vector.

The result should be a vector of 27x1, where the row indexes are the elements of domain. Column is the unkown variables.

This vector will be updated in every iteration, by adding the previous letter. New probabilities will be computed with respect to selected letter.

We need to compute it in a automatic way

The inference from future is not as trivial as forward inference.

There are two types of string pattern one is a... case. 

The other is a..b case. There is no such other case only the number of unkowns are changing.

I will identify the types of the strings with a function

For general case we can prove it with the following:


$ p(x_{1:N}|,x_0=\hat{x_0})$

$\alpha$

$p(x_{N+1}=\hat{x}_{N+1}|x_N)p(x_n|x_{n-1})\dots p(x_1|x_0=\hat{x}_0)$

The $p(x_1|x_0=\hat{x}_0)$ term can be obtained easily via row selection.

In order to obtain other terms we need to multiply T by N times. 

Then we need to take the transpoze of T.





In [22]:
########### Inference from Future START ###########  
def inference_from_future(first_char,last_char,N,T,start,l):
    prob = 1
    guess_table = np.zeros((27,N+2))
    x_0 = first_char
    x_prev = x_0
    word = np.zeros((0,1))
    #print("We have " + str(N) + " unkown ")
    if(first_char == ""):
        first_char = '.'
    #print("x_1"+ " = " + first_char)
    #print("x_"+str(N+1)+ " = " + last_char)
    word = np.zeros((0,N))
    word = np.vstack(first_char)
    for i in range(0,N):
        matrix = T
        for x_d in alphabet:
            #print("x_"+str(i + 2)+" = " + x_d ),
            guess_table[letter2idx[x_d],i] = T[letter2idx[first_char]][letter2idx[x_d]]
        #if l and i == 0:
            #for x_d in alphabet:
                #print("x_"+str(i + 2)+" = " + x_d ),
                #guess_table[letter2idx[x_d],i] = guess_table[letter2idx[x_d],i-1]
        #for j in range(0,N):
        if l:
            matrix=np.transpose(np.linalg.matrix_power(T,N-i)) # decrease the number of N since we pretend like we have found the letter
        else:
            matrix=np.transpose(np.linalg.matrix_power(T,N))
        guess_table[:,i] = matrix[letter2idx[last_char]]*guess_table[:,i]
        guess_table[:,i] = np.divide(guess_table[:,i],np.sum(guess_table[:,i]))
        if l:
            first_char = str(alphabet[np.argmax(guess_table[:,i])])
            prob = np.dot(prob,guess_table[letter2idx[first_char],i])
            # assign argmax as first char.
        else:
            first_char = str(np.random.choice(alphabet,1,p = guess_table[:,i])[0])
            word = np.vstack((word ,first_char)) 
        repeat = 1
    for i in range(0,repeat):
        for j in range(0,N):
            #if not l:
                #word = np.vstack((word ,str(np.random.choice(alphabet,1,p = guess_table[:,j])[0]))) 
            #if l:
                #print np.argmax(guess_table[:,j])
                word = np.vstack((word ,str(alphabet[np.argmax(guess_table[:,j])])))
        else:
            if not l:
                return word[start:]
            if l:
                return word[start:],prob
    
########### Inference from Future END ###########  
########### STRING OPERATION FUNCTIONS START ###########  
def underscore_finder(my_str):
    list_of_underscore_indices = []
    for i in range(0,len(my_str)):
        if(my_str[i] == "_"):
            list_of_underscore_indices.append(i)
    return list_of_underscore_indices
def group_problems(list_of_underscore_indices):
    problems = []
    for i in range(0,len(list_of_underscore_indices)):
        problems.append([])
    #print(problems)
    z = 0
    for i in range(0,len(list_of_underscore_indices)):
        problems[z].append(list_of_underscore_indices[i])
        if(i == len(list_of_underscore_indices)-1):
            break
        if(list_of_underscore_indices[i + 1] - list_of_underscore_indices[i] !=  1):
            z = z + 1
    #delete empty lists
    return [x for x in problems if x != []]
            
    
def problem_identify(my_str,list_of_underscore_indices,T,l):
    str_len = len(my_str)
    problems = group_problems(list_of_underscore_indices)
    #print(problems)
    p_num = len(problems)
    pb = 1
    #print("The amount of problem is : " + str(p_num))
    for i in range(0,p_num):
        p_type = 0
        #p_type = 1 if straight markov
        #p_type = 2 if inference from future is need
        #print("Working on problem no: "+ str(i))
        problem = problems[i]
        if(problem[0] == 0):
            prev = ""
        else:
            prev = my_str[problem[0]-1]
        if(problem[-1] == (len(my_str) -1) ):
            next_letter = ""
            p_type = 1
        else:
            next_letter = my_str[problem[-1]+1]
            p_type = 2
        #print "type = ", str(p_type),prev,len(problem)*"_",next_letter, " length ",len(problem)
        if not l:
            my_str = inference(p_type,my_str,problem,prev,next_letter,T,l)
        else:
            my_str,prob = inference(p_type,my_str,problem,prev,next_letter,T,l)
            pb = np.dot(prob,pb)
    if l:
        print "Log prob:", np.log(pb)
    return my_str
########### STRING OPERATION FUNCTIONS END ########### 

def inference(p_type,my_str,indices,prev_letter,next_letter,T,l):
    prob = 1
    if(my_str == ""):
        #print("Null string error")
        return ""
    if(p_type == 1):
        #Straight markov
        #print("Problem type = 1")
        if prev_letter == "":
            result = straight_markov(first_char=".",T=T,N=len(indices),repeat=1,start=1,p=False,l=l)
            #print("Straight! 1")
            #print(result)
            a = 0
            for i in indices:
                list1 = list(my_str)
                list1[i] = str("".join(result[a]))
                a = a+1
                my_str = "".join(list1)
            #print "Result is ", my_str
        else:
            result = straight_markov(first_char=prev_letter,T=T,N=len(indices),repeat=1,start=1,p=False,l=l)
            #print("Straight! 2")
            #print(result)
            a = 0
            for i in indices:
                list1 = list(my_str)
                list1[i] = str("".join(result[a]))
                a = a+1
                my_str = "".join(list1)
            #print "Result is ", my_str
    elif(p_type == 2):
        if l:
            result,pb = inference_from_future(first_char = prev_letter,last_char = next_letter,N=len(indices),T=T,start = 1,l=l)
            prob = np.dot(pb,prob)
        else:
            result = inference_from_future(first_char = prev_letter,last_char = next_letter,N=len(indices),T=T,start = 1,l=l)

        #print("From future!")
        #print(result)
        a = 0
        for i in indices:
            list1 = list(my_str)
            list1[i] = str("".join(result[a]))
            a = a+1
            my_str = "".join(list1)
        #print "Result is ", my_str
    if l:
        return my_str,prob
    return my_str


In [26]:
for i in range(0,len(test_strings)):
    #repeat samples
    print "samples for " ,test_strings[i]
    for j in range(0,10):
        print(problem_identify(test_strings[i],underscore_finder(test_strings[i]),T,False))
#group_problems(underscore_finder(test_strings[0]))
#print(test_strings[1])
#problem_identify(test_strings[1])


samples for  th__br__n.f_x.
th.ybrm.n.fex.
theqbrorn.fex.
therbrzzn.fex.
theybrewn.fix.
therbr.fn.fex.
th.mbrhen.f.x.
theabrr.n.fix.
thenbr.pn.fex.
thecbry.n.fex.
thedbresn.fix.
samples for  _u_st__n_.to_be._nsw_r__
auesthend.tombe.answera.
ou.sthyns.to.be.insward.
ourstidnk.to.be.answerre
ourst.ine.toube.inswer.s
tulstinnd.to.be.onsworve
susst.tnd.toube.answer.o
oursthent.to.be.answirri
oursthond.toube.inswarmi
bu.stornd.to.be..nswere.
ousst.bng.tombe.inswere.
samples for  i__at_._a_h_n_._e_r_i_g
iluata.mathent.seprling
iofaty.mathang.bepr.ing
insath..ashing.me.r.ing
indats.cathind.be.rging
idsate.mathiny.wedr.ing
im.atr.bathins.ye.r.ing
iouaty.hathing.seereing
ingatt.pathand.sefr.igg
inoath.mathens.merrting
it.at..jathind.le.r.ing
samples for  q___t.___z._____t.__.___.__.
qungt.ff.z.h.tint.ti.lec.e..
qugrt.wo.z.by.wot.m..oou.an.
qutht.jonz.tidint.ar.hen.ar.
qur.t.nd.z.why.ct.hu.men.pl.
qu.pt.catz..eed.t.th.hte.wh.
qulet.thaz.tinont.is.med.cr.
qunot.picz.h.thit.ot.the.be.
quget.houz.d

### Part 3
Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.
My algorithm is like:

Compute the probability for the first unknown element

Take the maximum element of the probability distribution (one dimensional). Update the table with respect to new element.

Assign the found element as first element. Decrease the number of multiplication by one. N--

Continue until end.


In [27]:
for i in range(0,len(test_strings)):
    print(problem_identify(test_strings[i],underscore_finder(test_strings[i]),T,True))

Log prob: -3.07433486577
the.br.an.fex.
Log prob: -8.32284514738
oursthend.to.be.answere.
Log prob: -11.636089996
in.ath.wathend.he.r.ing
Log prob: -22.9236424228
qur.t.thiz.the.at.an.the.an.


### Part 4
Argmax is a good way to predict most likely sequence of strings. However, it is too much constrained, because we dont use other probabilities which may be also more likely. In order to check that we should generate all the possible strings in the domain. Then compute the each of the probabilities of the strings. Then, we can check the difference between the argmax, and the second string comes from argmax. If the difference is very less, our program continue to check the difference between the next string by popping each. By following this idea we may 


For further analysis.We can use factor graph message passing algorithm. That way it will provide better results to this problem. The algorithm will be less complicated. Since it supports more complicated forms of graphs. Thanks to that flexibility we can connect 

